In [10]:
import tensorflow as tf
import time
import numpy as np
from models import tfBoW
from collections import defaultdict

In [4]:
# read data
# read corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]

def read_dataset(file):
    with open(file, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in words.split(" ")], t2i[tag])


In [5]:
train = list(read_dataset("../data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/train.txt"))
nwords = len(w2i)
ntags = len(t2i)
print("Vocab size: {}".format(nwords))
print("No of tags: {}".format(ntags))

Vocab size: 16582
No of tags: 5


In [6]:
# create model
model = tfBoW(nwords, ntags)
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [8]:
@tf.function(experimental_relax_shapes=True)
def train_step(words, tag):
    with tf.GradientTape() as tape:
        scores = model(words)
        loss = loss_fn(tag, scores)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

In [11]:
for epoch in range(100):
    start_time = time.time()
    train_loss = 0.0
    for words, tag in train:
        words = tf.constant(words, dtype=tf.float32)
        tag = tf.constant([tag], dtype=tf.float32)
        loss = train_step(words, tag)
#     loss_ = train_step(words, tag)
    train_loss += loss.numpy()
    print("Epoch %r: train loss/sent=%.4f, time=%.2fs" % (
                epoch, train_loss/len(train), time.time()-start_time))
    test_correct = 0
    for words, tag in dev:
        words = tf.constant(words, dtype=tf.float32)
        tag = tf.constant([tag], dtype=tf.float32)
        scores = model(words)[0].numpy()
        predict = np.argmax(scores)
        if predict == tag:
            test_correct += 1
    print("epoch %r: test acc=%.4f" % (epoch, test_correct/len(dev)))
    

Epoch 0: train loss/sent=329.7098, time=24.58s
epoch 0: test acc=0.1996
Epoch 1: train loss/sent=329.7070, time=26.55s
epoch 1: test acc=0.1948
Epoch 2: train loss/sent=329.7128, time=29.41s
epoch 2: test acc=0.1966
Epoch 3: train loss/sent=329.7158, time=26.76s
epoch 3: test acc=0.2101
Epoch 4: train loss/sent=329.7070, time=29.29s
epoch 4: test acc=0.2024
Epoch 5: train loss/sent=329.7039, time=21.09s
epoch 5: test acc=0.1969
Epoch 6: train loss/sent=329.7064, time=21.04s
epoch 6: test acc=0.1951
Epoch 7: train loss/sent=329.7057, time=21.41s
epoch 7: test acc=0.2037
Epoch 8: train loss/sent=329.7081, time=25.65s
epoch 8: test acc=0.1898
Epoch 9: train loss/sent=329.7061, time=23.86s
epoch 9: test acc=0.1825
Epoch 10: train loss/sent=329.7057, time=24.55s
epoch 10: test acc=0.1997
Epoch 11: train loss/sent=329.7071, time=23.67s
epoch 11: test acc=0.1910
Epoch 12: train loss/sent=329.7060, time=27.20s
epoch 12: test acc=0.1875
Epoch 13: train loss/sent=329.7058, time=25.40s
epoch 13: 